In [1]:
%%javascript
$('<style>#appmode-leave{display:none}</style>').appendTo('head');

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as w
import nanohub.remote as nr
import os

def FileLink( filepath, path="files"):
   global jupyter_notebook_url
   import os
   if os.path.exists(filepath):
       bp = os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID'])
       ap = os.path.abspath( filepath )
       if ap.startswith(bp):
           link =  "/".join(jupyter_notebook_url.split("/",8)[:7])
           link += "/" + path + "/" + os.path.relpath(ap,bp)
           return link
       else:
           raise Exception("Dont have access to the file")
   else:
       raise Exception(filepath + " does not exists")
        


In [3]:
auth_data = {'grant_type' : 'tool'}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
session = nr.Session(auth_data, url='https://cdmhub.org/api/')
req_json = session.requestGet('/projects/list?verbose=1')
req_json = req_json.json()
projects = {}
for p in req_json['projects']:
    #if 'a3dconsortium' in p['alias']:
    if 'cams' in p['alias']:
        path = '/data/projects/' + p['alias'] + "/files"
        if os.path.exists(path):
            p['path'] = path
            projects[p['alias']] = p
req_json = session.requestGet('/members/' + str(os.getuid()))
data = req_json.json()
data = data['profile']

In [4]:
resultsimage = w.Image(value = open("SimResults.png", 'rb').read(), height=480)
resultsbutton = w.HTML(value='''
   <a target="blank" href="'''  + FileLink('a3dWizard.ipynb', "apps") + '''">
   <button style='background-color:#2B5E91;
   border:none;color:white;
   border-radius: 6px;
   padding:6px;
   text-align: center;
   text-decoration:none;
   display:inline-block;
   font-size: 16px;
   margin: 4px 2px;
   width:99%'>
   View Simulation Results
   </button></a>
''')
results = w.VBox(
    [
        resultsimage,
        resultsbutton
    ],
    layout = w.Layout(width="50%")
)


In [5]:
jobsimage = w.Image(value = open("ProcessSims.png", 'rb').read(), height="480")
jobsbutton = w.HTML(value='''
   <a target="blank" href="'''  + FileLink('SubmitJobs.ipynb', "apps") + '''">
   <button style='background-color:#2B5E91;
   border:none;color:white;
   border-radius: 6px;
   padding:6px;
   text-align: center;
   text-decoration:none;
   display:inline-block;
   font-size: 16px;
   margin: 4px 2px;
   width:99%'>
   Create New Process Simulation
   </button></a>
''')
jobs = w.VBox(
    [
        jobsimage,
        jobsbutton
    ],
    layout = w.Layout(width="50%")
    
)


In [13]:
user = w.HTML(value='''
   <div style='background-color:#FFF;
   border-radius: 6px;
   border: 1px solid #DDD;
   padding:6px;
   text-align: left;
   text-decoration:none;
   display:inline-block;
   margin: 4px 2px;
   width:99%;
   height:99%'>
   <img src=''' + data['picture']['thumb'].replace("https://cdmhub.org/","") + ''' width="200" height="200"></img>
   <p style='padding:20px;border: 1px solid #DDD;'>'''+data['name']+'''</br>
   '''+data['organization']+'''</br>
   '''+data['email']+'''</p>
   </div>
''')

In [7]:
children = []
colors = ['#912b5e','#915e2b','#2b915e','#2b5e91']
c=0
for i, p in projects.items():
    children.append(w.HTML(value='''
        <div style='
        background-color:#FFF;
        border:1px solid #AAA;
        color:white;
        border-radius: 6px;
        padding:0px;
        margin:10px
        text-align: left;
        text-decoration:none;
        display:inline-block;
        font-size: 16px;
        margin: 4px 2px;
        min-width:320px;width:320px; display:flex'>
        <div style="flex:1;padding:10px"><a target="blank" href="'''  + projects[i]['uri'] + '''">''' + projects[i]['title'] +'''</a></div>
        <div style="background-color:''' + colors[c % 4] + ''';width:80px" >&nbsp;</div>
</div>
'''))
    c+=1

In [8]:
dividerproject = w.HTML(value='''
   <div style='background-color:#2B5E91;
   border:none;color:white;
   border-radius: 6px;
   padding:6px;
   text-align: left;
   text-decoration:none;
   display:inline-block;
   font-size: 16px;
   margin: 4px 2px;
   width:99%'>
   <img src=''' + FileLink('./Additive3D.png') + ''' width="32"></img>
   PROJECTS
   </div>
''')

In [9]:
dividermodule = w.HTML(value='''
   <div style='background-color:#2B5E91;
   border:none;color:white;
   border-radius: 6px;
   padding:6px;
   text-align: left;
   text-decoration:none;
   display:inline-block;
   font-size: 16px;
   margin: 4px 2px;
   width:99%'>
   <img src=''' + FileLink('./Additive3D.png') + ''' width="32"></img>
   MODULES
   </div>
''')

In [10]:
dividerheader = w.HTML(value='''
   <div style='background-color:#2B5E91;
   border:none;color:white;
   border-radius: 6px;
   padding:6px;
   text-align: left;
   text-decoration:none;
   display:inline-block;
   font-size: 16px;
   margin: 4px 2px;
   width:99%'>
   <img src=''' + FileLink('./logoA3D.png') + ''' width="200"></img>
   </div>
''')

In [11]:
dividerfooter = w.HTML(value='''
   <div style='background-color:#FFFF;
   color:#000;
   border:1px solid #AAA;
   border-radius: 6px;
   padding:0px;
   text-align: center;
   text-decoration:none;
   display:inline-block;
   font-size: 14px;
   margin: 4px 2px;
   display:flex;
   width:99%'>
       <div style="flex:1"><a target="blank" href="https://cdmhub.org/groups/camsconsortium/forum/suggestions-for-additive3d/faq---additive3d"><img src=''' + FileLink('./FAQ.png') + ''' width="40"></img>FAQ</a></div>
       <div style="flex:1"><a target="blank" href="https://cdmhub.org/groups/camsconsortium/forum/suggestions-for-additive3d/report-technical-issues"><img src=''' + FileLink('./TechnicalIssuesLogo.png') + ''' width="40"></img>Report Technical Issues</a></div>
       <div style="flex:1"><a target="blank" href="https://cdmhub.org/groups/camsconsortium/collections"><img src=''' + FileLink('LiteratureLogo.png') + ''' width="40"></img>Technical Resources</a></div>
   </div>
''')
dividerfooter

HTML(value='\n   <div style=\'background-color:#FFFF;\n   color:#000;\n   border:1px solid #AAA;\n   border-ra…

In [14]:

box_layout = w.Layout(overflow='scroll hidden',
                    width='100%',
                    height='',
                    display='flex')
w.VBox([
    dividerheader,
    w.HBox([
        user,
        w.VBox([
            dividermodule,
            w.HBox(children),
            dividerproject,
            w.HBox([jobs,results], layout=box_layout)
        ])
    ]),
    dividerfooter
])